In [ ]:
#HCD_ndar_edinburgh_2022release (Lifespan 3.0)
#add extra export statement to put race and ethnicity of parents in separate file for socdem01 pickup.

In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
config = LoadSettings()


In [ ]:
box=LifespanBox(cache='./')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
pathout="./prepped/hcd" 
racepath="./prepped"

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
extrainfo=config['rosetta']['filename']

eventlist=['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1','visit_arm_1']

In [ ]:
inventory=pd.read_csv(extrainfo)
inventory.columns
inventory=inventory.loc[inventory.nda_age.isnull()==False]

Fullinventory=inventory.copy()
print("Full Inventory w All events:",Fullinventory.shape)

In [ ]:
inventory.loc[inventory.nda_age>300][['nda_age','subject','event_age']]

In [ ]:
#extract the inventory subset 
inventory=pd.read_csv(extrainfo)[['DB_Source','REDCap_id_parent','subject','parent_at_visit', 'parent_at_V1','ParentPIN','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event','REDCap_id','race','ethnic_group','site']]
inventory=inventory.loc[inventory.nda_age.isnull()==False]
inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
inventory.nda_age=inventory.nda_age.round(0).astype(int)
print('Inventory Shape out of the Box',inventory.shape)

#These subjects can't go to NDA.  
#We don't have any of the necessary NDA variables for these ParentPIN subjects who showed up for NIH Toolbox sessions but not REDCap
inventory.loc[~(inventory.parent_at_visit==inventory.ParentPIN.str[0:10]) & (inventory.ParentPIN.isnull()==False)]

p=inventory.loc[inventory.parent_at_V1.isnull()==False]
print("Number of Unique Subjects",len(inventory.subject.unique()))
print("Number of Unique Parents",len(p.parent_at_V1.unique()))  

##these guys, too, have minimal information.  
##will place all toolbox parent about self data under parent_at_V1 or omit completely from NDA push
##inventory.loc[~(inventory.parent_at_visit==inventory.parent_at_V1) & (inventory.redcap_event_name.isin(['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1']))]
sublist=list(inventory.subject.unique())+list(p.parent_at_V1.unique())
d=pd.DataFrame(sublist,columns=['subject'])
d.subject.head()

d=d.loc[~(d.subject.isnull()==True)]
subjectlist=list(d.subject)
len(subjectlist)
print('Number of parent_at_V1+subjects not NAN',len(subjectlist))

print('inventory shape',inventory.shape)
inventory=inventory.loc[~(inventory.subject.isnull()==True)]
print('inventory shape',inventory.shape)

print('inventory shape',inventory.shape)
p=inventory.loc[~(inventory.parent_at_V1.isnull()==True)]
print('inventory shape',p.shape)


## Prepare ped file for merging with ndar subjects (first parents, then children and teens)

In [ ]:
#curated pedigrees contain redundant information with inventory (e.g. pedids and pseudoguids, HCA_ID, HCD_ID, etc).
#however need to keep sex of parents.  
peds=883930709978
pathpeds=box.downloadFile(peds)
pedids=pd.read_csv(pathpeds)
pedids=pedids.drop(columns=['Familyflag','HCD_ID','HCA_ID','site','parent','age','relationship1', 'with1', 'relationship2', 'with2', 'relationship3',
       'with3', 'relationship4', 'with4', 'relationship5', 'with5', 'src_avuncular1_id','src_grandparent1_id','src_cousin_id1','src_cousin_id2',
       'relationship6', 'with6', 'relationship7', 'with7'])
pedids.loc[pedids.subject.str.contains('HCD')].head()
pedids.loc[pedids.subject.str.contains('HCD')].shape

#these are all second parents
pedids.loc[~(pedids.subject.isin(subjectlist)) & (pedids.subject.str.contains('HCD'))]
pedids.shape
#pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isnull()==False)]


In [ ]:
#these special cases (unspecified half sibs, and twins) need to be noted in release notes somewhere - already noted in family flag internally
#e.g. subj is list of people who have non-conforming relationship type w.r.t siblings that NDA can't handle
subj=list(pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['HS','FTS','FTB'])),'subject']) #HCD0295146,HCD2685169,HCD2971776,HCD1197757
#['HCD0295146->FB', 'HCD2685169->FS', 'HCD2971776->HS', 'HCD1197757->HS']
pedids['zygosity']=''
pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['HS'])),'sibling_type1']='HMS'
pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['FTB'])),'sibling_type1']='FB'
pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['FTB'])),'zygosity']='dizygous'

pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['FTS'])),'sibling_type1']='FS'
pedids.loc[(pedids.subject.str.contains('HCD')) & (pedids.sibling_type1_alt.isin(['FTS'])),'zygosity']='dizygous'

#pedids.loc[pedids.subject.isin(subj)]


In [ ]:
#now deal with adoptive parents to get themm into NDA format
pedids2=pedids.rename(columns={'biomomsubj':'src_mother_id','biodadsubj':'src_father_id'}).copy()
adopts=pedids2.loc[(pedids2.adoptmomsubj.isnull()==False)|(pedids2.adoptdadsubj.isnull()==False)].copy()# & (pedids.biomomsubj=='0')]
adopts.head(2)
adoptsubj=list(adopts.subject)
#adoptsubj
#['HCD0236130',
# 'HCD0486254',
# 'HCD0406230',
# 'HCD0977576',
# 'HCD0822850',
# 'HCD1021821',
# 'HCD0075435',
# 'HCD2491257',
# 'HCD2502640',
# 'HCD0039128',
# 'HCD2044333',
# 'HCD0146331',
# 'HCD1170131',
# 'HCD1331432',
# 'HCD0868268']
pedids2.loc[(pedids2.adoptmomsubj.isnull()==False) & (pedids2.src_mother_id=='0'),'src_mother_id']=pedids2.adoptmomsubj
pedids2.loc[(pedids2.adoptdadsubj.isnull()==False) & (pedids2.src_father_id=='0'),'src_father_id']=pedids2.adoptdadsubj
pedids2.loc[(pedids2.adoptmomsubj.isnull()==False) & (pedids2.src_mother_id.str.contains('DMOM')),'src_mother_id']=pedids2.adoptmomsubj
pedids2.loc[(pedids2.adoptdadsubj.isnull()==False) & (pedids2.src_father_id.str.contains('DDAD')),'src_father_id']=pedids2.adoptdadsubj

pedids2.loc[(pedids2.subject.isin(adoptsubj)) & (pedids2.src_father_id=='0'),'src_father_id']='DDAD1_'+ pedids2.pedid.astype(str) #,'src_father_id']=pedids.
pedids2.loc[(pedids2.subject.isin(adoptsubj)) & (pedids2.src_mother_id=='0'),'src_mother_id']='DMOM1_'+ pedids2.pedid.astype(str) #,'src_father_id']=pedids.

pedids2.loc[(pedids2.subject=='HCD0486254') ,'src_father_id']='DDAD2_5073'
pedids2.head()

pedids2.loc[pedids2.subject.str.contains('HCD')].shape

#s=pd.DataFrame(subjectlist,columns=['subject'])
#s.head()
#peds3=pd.merge(s,pedids2,on='subject',how='outer',indicator=True)
#peds3._merge.value_counts()#head()
#peds3.loc[peds3._merge=='left_only']
#peds3.subject.value_counts()
pedids3=pedids2.loc[pedids2.subject.isin(subjectlist)]
pedids3.loc[pedids3.subject.str.contains('HCD')].shape
#pedids3.loc[pedids3.subject==''].shape
#pedids3.shape
#pedids2.shape
#len(subjectlist)
#for i in subjectlist:
#    print(i)  
    #if 'HCD' not in i:
    #    print(i)
    #else:
    #    print(i,"wierd")
        
#subjectlist.to_csv('test.csv',index=False)

## First Deal with Parents; Get them in to ndar_subjets format for stacking with children and teens)

In [ ]:

parentinventory=inventory.drop(columns=['parent_at_visit','race','ethnic_group','pedid','M/F','nda_age','pseudo_guid','ParentPIN']).rename(columns={'subject':'comqother'})
parentinventory.comqother="Parent about self as context for " + parentinventory.comqother
parentinventory=parentinventory.rename(columns={'parent_at_V1':'subject'})
print(parentinventory.shape)

#p=      inventory.loc[~(inventory.parent_at_V1.isnull()==True)]
p=parentinventory.loc[~(parentinventory.subject.isnull()==True)]
print('parentinventory #unique parents',len(p.subject.unique()))

parentinventory=parentinventory.loc[parentinventory.DB_Source.isin(['parent_only','parentandchild'])]
alt=parentinventory.loc[parentinventory.subject.isin(list(p.subject.unique()))]

print('DB_Source counts',parentinventory.shape)
print('Alt counts',alt.shape)

parentinventory.head()
parentinventory.loc[parentinventory.subject.isnull()==True]
a=parentinventory.loc[~(parentinventory.subject.isnull()==True)]
print('parentinventory shape',len(a.subject.unique()))


In [ ]:
pedids3.head()
#parentinventory.head()
pedids3.shape

In [ ]:
#merge ped vars with inventory (inventory already has pedid) by parent_at_V1 to get the parents some data
pedids3=pedids3.drop(columns=['adoptmomsubj','adoptdadsubj','sibling_type1_alt'])

pedids3.head()
print('parentinventory shape before',parentinventory.shape)
parentinventory2=pd.merge(parentinventory,pedids3,left_on=['subject'],right_on=['subject'],how='inner')
print('parentinventory shape after',parentinventory2.shape)
parentinventory2.loc[parentinventory2.psuedo_guid=='NDAR_INVGR630MPG']
print("number of unique parents",len(parentinventory2.subject.unique()))


In [ ]:
#get parent race ethnicity and ages.  Use pseudo_guid and sex from pedid because otherwise will only grab from V1
#ndar subjects needs required vars for regular and NIH Toolbox guest appearance parents
hcdparent=RedcapTable.get_table_by_name('hcpdparent').get_frame(['parent_age','redcap_event_name','id'])
print(hcdparent.describe())
dfnewparent=hcdparent.copy()
dfnewparent.head()
#hcdparent=RedcapTable.get_table_by_name('hcpdparent').get_frame(['parent_age','redcap_event_name','id','p_race','p_latino'])


In [ ]:
#parentinventory2.columns
dfnewparent.columns
print(dfnewparent.shape)
dfnewparent.drop_duplicates(subset=['id','redcap_event_name']).shape

print("number of unique parents",len(parentinventory2.subject.unique()))
parentinventory2.columns


In [ ]:
#assign missing values for ages
#store mean for missing values after inference
meanparentageV1=pd.DataFrame(dfnewparent.loc[dfnewparent.redcap_event_name=='visit_1_arm_1','parent_age'].describe()).loc['mean'][0].round(0)

#merging by id assures that we're just going with parent_at_V1 for purpose of GUIDS and ages.  Other parents dont have REDCap records. 
#not that the shape reduction of 78 observations corresponds with covid_2_arm_1 and followup_arm_1 with empty values for id

inventory2=pd.merge(parentinventory2,dfnewparent,left_on=['REDCap_id_parent','redcap_event_name'],right_on=['id','redcap_event_name'],how='inner')
inventory2[['subject']]
print('Inventory2 shape after merging with REDCap',inventory2.shape)

#find those with missing ages
p1=inventory2.loc[(inventory2.parent_age.isnull()==True)]# & (inventory2.redcap_event_name.isin(['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1']))]

#get those with missing ages and their NDA variables
p3=inventory2.loc[inventory2.id.isin(list(p1.REDCap_id_parent))][['redcap_event_name','id','parent_age','nda_interview_date']]
#p3['nda_interview_date']=pd.to_datetime(p3['nda_interview_date'])
p4=p3.sort_values(by=['id','nda_interview_date']).copy()
p4.head()
#inventory2.head()
#inventory2.shape
p4.shape
inventory2.columns

In [ ]:
inventory2['nda_interview_date']=pd.to_datetime(inventory2['nda_interview_date'],errors='coerce')
inventory2.nda_interview_date.dt.strftime('%Y').astype(int)
inventory2.nda_interview_date.value_counts()

In [ ]:
#fill in missing ages

p6=pd.DataFrame()
for sub in list(inventory2.REDCap_id_parent.unique()):
    #print(sub)
    p4=inventory2.copy()
    p5=p4.loc[p4.REDCap_id_parent==sub].copy()
    p5['oldage']=p5.parent_age
    #initialize
    p5['newage']=0.0
    p5['years']=p5.nda_interview_date.dt.strftime('%Y').astype(int)
    p5['lastage']=p5.parent_age.shift(+1)
    p5['lastyear']=p5.years.shift(+1)
    p5['nextage']=p5.parent_age.shift(-1)
    p5['nextyear']=p5.years.shift(-1)
    #iterate forward and backwards a few times to fill in missing ages when it can be inferred from other visits
    for x in range(0, 4):
        #forward
        p5.loc[(p5.parent_age.isnull()==True),'newage']=p5.lastage + (p5.years-p5.lastyear)
        p5.loc[p5.parent_age.isnull()==True,'parent_age']=p5.newage
        p5['lastage']=p5.parent_age.shift(+1)
        p5['lastyear']=p5.years.shift(+1)
        #p5.loc[p5.parent_age.isnull()==True,'newage']=p5.lastage + (p5.years-p5.lastyear)
        #p5.loc[p5.parent_age.isnull()==True,'parent_age']=p5.newage       
        #backwards
        p5.loc[p5.parent_age.isnull()==True,'newage']=p5.nextage - (p5.nextyear-p5.years)
        p5.loc[p5.parent_age.isnull()==True,'parent_age']=p5.newage
        p5['nextage']=p5.parent_age.shift(-1)
        p5['nextyear']=p5.years.shift(-1)
    #initialize with the mean for any subjects still missing ages
    for x in range(0,1):
        #print(sub)
        p5.loc[(p5.parent_age.isnull()==True) & (p5.redcap_event_name=='visit_1_arm_1'),'newage']=meanparentageV1
        p5.loc[(p5.parent_age.isnull()==True) & (p5.redcap_event_name=='visit_1_arm_1')==True,'parent_age']=p5.newage
        p5['lastage']=p5.parent_age.shift(+1)
        p5['lastyear']=p5.years.shift(+1)
        p5.loc[(p5.parent_age.isnull()==True),'newage']=p5.lastage + (p5.years-p5.lastyear)
        p5.loc[p5.parent_age.isnull()==True,'parent_age']=p5.newage
    p6=pd.concat([p6,p5],axis=0)
 
p6['parent_nda_age']=12 * p6.parent_age
print(p6.shape)
inventory2.columns


In [ ]:
p6.loc[p6.parent_age.isnull()==True]
p6.shape
#p6.head()
#inventory2.shape

In [ ]:
p7=p6[['redcap_event_name','id','parent_age']].rename(columns={'id':'REDCap_id_parent'})
inventory3=pd.merge(inventory2,p7,how='left',on=['REDCap_id_parent','redcap_event_name'])
print('together ages:',inventory3.shape)
#
inventory3.loc[(inventory3.parent_age_x.isnull()==True) & (inventory3.redcap_event_name),'parent_age_x']=inventory3.parent_age_y
inventory3=inventory3.rename(columns={'parent_age_x':'parent_age'}).drop(columns=['parent_age_y'])
inventory3.columns
print('merge again:',inventory3.shape)

inventory3=inventory3.drop(columns=['id'])
#rename back to orig for next stesp
inventory=inventory3.copy()
inventory.columns
#inventory.shape
#inventory.head()
inventory.loc[inventory.parent_age.isnull()==True]
print('one more check:',inventory3.shape)


This is the method used to derive children's categorical races from checkbox varialbe in the inventory ([config][rosetta]). Implies that someone self-reporting as Asian and White is Multiple races.  This is a limitation, as there are many individuals who would check 'Asian' on a categorical variable but 'Asian and White' on a checkbox variable.  Distributional consequences.

Child races have already been calculated and stored in inventory.  Parental races not calculated for the inventory, so added here.

12, NATIVE AMERICAN | 18, ASIAN |  11, BLACK/AFRICAN AMERICAN |  14, NATIVE HAWAIIAN or
#OTHER PACIFIC ISLANDER |10, WHITE | 25, MORE THAN ONE RACE |  99, DON'T KNOW


In [ ]:
#get races
racevars=RedcapTable.get_table_by_name('hcpdparent').get_frame(['redcap_event_name','parent_id','child_id','id','p_race','p_latino'])
racevars=racevars.loc[racevars.redcap_event_name=='visit_1_arm_1'].copy()
racevars.head()
#print(racevars.shape)

In [ ]:
racevars['p_sumcheckedrace']=racevars[['p_race___10','p_race___11','p_race___12','p_race___14','p_race___18','p_race___25']].sum(axis=1)
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___10==1),'p_racial']=5
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___11==1),'p_racial']=3
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___12==1),'p_racial']=1
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___14==1),'p_racial']=4
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___18==1),'p_racial']=2
racevars.loc[(racevars.p_sumcheckedrace==1) & (racevars.p_race___25==1),'p_racial']=6
racevars.loc[(racevars.p_sumcheckedrace==0) & (racevars.p_race___99==1),'p_racial']=99
racevars.loc[racevars.p_sumcheckedrace>=2,'p_racial']=6

racevars.loc[racevars.p_racial.isnull()==True,'p_racial']=99
racevars.loc[racevars.p_latino.isnull()==True,'p_latino']=9

racevars.p_racial
racevars.p_racial.value_counts(dropna=False)
#racevars.p_sumcheckedrace.value_counts(dropna=False)
racevars.p_latino.value_counts(dropna=False)

In [ ]:
racevars['p_race']=racevars.replace({'p_racial':
                                       {1.0:'American Indian/Alaska Native',
                                        2.0:'Asian',
                                        3.0:'Black or African American',
                                        4.0:'Hawaiian or Pacific Islander',
                                        5.0:'White',
                                        6.0:'More than one race',
                                        99.0:'Unknown or not reported'}})['p_racial']
racevars['p_ethnic_group']=racevars.replace({'p_latino':
                                           {1.0:'Hispanic or Latino',
                                            0.0:'Not Hispanic or Latino',
                                            9.0:'Unknown or not reported'}})['p_latino']

In [ ]:
racevars.p_race.value_counts(dropna=False)
#racevars.p_ethnic_group.value_counts(dropna=False)
racevars.columns
print(Fullinventory.columns) #inventory only has parents, and you need child subject for merge
print(racevars.columns)

In [ ]:
#write out csv with parental race and ethnicity information for pulling into socdem
socdem=racevars[['child_id', 'p_race','p_ethnic_group', 'parent_id']].copy() # 'subjectkey','interview_date', 'interview_age','site',
extras=Fullinventory.loc[Fullinventory.redcap_event=='V1'][['DB_Source','subject','nda_age','race','ethnic_group','parent_at_V1','redcap_event_name']]
socdem=pd.merge(socdem,extras,left_on='child_id',right_on='subject',how='right')#,indicator=True)
socdem.DB_Source.value_counts()
socdem=socdem.rename(columns={'subject':'src_subject_id','p_race':'cg1_race','p_ethnic_group':'cg1_ethnicity','ethnic_group':'ethnicity'})
socdem.loc[(socdem.cg1_race.isnull()==True) & (socdem.nda_age.astype(int) < 216)][['cg1_race']]#='Unknown or not reported'
socdem.to_csv(racepath+'/HCPD_racethnic_for_socdem01_' +snapshotdate+'.csv',index=False)
#socdem.shape

#Fullinventory.loc[Fullinventory.redcap_event=='V1'].shape

In [ ]:

racevars=racevars.drop(columns=['p_latino',
       'p_race___10', 'p_race___11', 'p_race___12', 'p_race___14',
       'p_race___18', 'p_race___25', 'p_race___99', 
        'p_sumcheckedrace','p_racial','redcap_event_name'])

racevars=racevars.rename(columns={'p_race':'race','p_ethnic_group':'ethnic_group'})
racevars.head()



In [ ]:
parentinventoryfinal=pd.merge(inventory,racevars,left_on='REDCap_id_parent',right_on='id',how='left')

parentinventoryfinal['M/F']=parentinventoryfinal.replace({'sex':
                                           {1.0:'M',
                                            2.0:'F',
                                            }})['sex']

parentinventoryfinal.sex.value_counts(dropna=False)
parentinventoryfinal['M/F'].value_counts(dropna=False)
#nda age in months
parentinventoryfinal['nda_age'] = 12 * parentinventoryfinal.parent_age

parentinventoryfinal.head()

### parentinventoryfinal (above) is the frame to stack with children for ndar subject

# Now create the child/teen data frame, which will have ndar variables as well as edinburgh variables


In [ ]:
pedids3.columns
extrainfo

In [ ]:
childinventory=Fullinventory

print(childinventory.shape)
childinventory=childinventory.loc[childinventory.DB_Source.isin(['child_only','teen','parentandchild'])]
print(childinventory.shape)
childinventory=childinventory.loc[childinventory.nda_age.isnull()==False]
print(childinventory.shape)
#inventory.nda_interview_date=pd.to_datetime(inventory.nda_interview_date).dt.strftime('%m/%d/%Y')
#inventory.nda_age=inventory.nda_age.round(0).astype(int)
childinventory.columns

In [ ]:
pedids3.columns

# Cat children/teens and parents together for NDAR

In [ ]:
#merge with pedids3
childinventory2=pd.merge(childinventory.drop(columns='pedid'),pedids3,how='left',on='subject')#,indicator=True)
#print(childinventory2._merge.value_counts())
#print(childinventory2.columns)
#print(parentinventoryfinal.columns)

keeplist=['DB_Source','subject','nda_interview_date','nda_age','M/F',
          'redcap_event_name','redcap_event','race','ethnic_group',
          'site', 'psuedo_guid', 'pedid','src_mother_id',
          'src_father_id','src_sibling1_id',
          'sibling_type1', 'src_sibling2_id',
          'sibling_type2', 'src_sibling3_id', 'sibling_type3', 'zygosity']

childinventory3=childinventory2[keeplist+['REDCap_id']].copy()
parentinventory3=parentinventoryfinal[keeplist].copy()
#print('childinventoryshape',childinventory3.shape)
#print('parentinventoryshape',parentinventory3.shape)

c=childinventory3.reset_index().copy()
p=parentinventory3.reset_index().copy()
ndarinventory=pd.concat([c,p],axis=0)
#print(ndarinventory.shape)
ndarinventory=ndarinventory.rename(columns={"subject":"src_subject_id","M/F":"gender","nda_age":"interview_age",
                           "nda_interview_date":"interview_date",
                           "pedid":"family_user_def_id","psuedo_guid":"subjectkey",'redcap_event':'visit'})

ndarinventory['phenotype']=pd.Series(("Healthy Subject" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory['phenotype_description']=pd.Series(("In good health" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory['twins_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory['sibling_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory['family_study']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)
ndarinventory['sample_taken']=pd.Series(("No" for i in range(ndarinventory.shape[0])),index=ndarinventory.index)


ndarinventory=ndarinventory.loc[ndarinventory.redcap_event_name.isin(eventlist)]
ndarinventory=ndarinventory.drop(columns=['DB_Source','REDCap_id','redcap_event_name'])
print(ndarinventory.columns)

In [ ]:
ndarinventory.interview_date=pd.to_datetime(ndarinventory.interview_date).dt.strftime('%m/%d/%Y')
ndarinventory.interview_age=ndarinventory.interview_age.round(0).astype(int)
siblist=['sibling_type1','sibling_type2','sibling_type3']
for ii in siblist:
    ndarinventory[ii]=ndarinventory.replace({ii:
                                       {'FB':'Full Brother FB',
                                        'HMB':'Half Mother Brother HMB',
                                        'HFB':'Half Father Brother HFB',
                                        'FS':'Full Sister FS',
                                        'HMS':'Half Mother Sister HMS',
                                        'HFS':'Half Father Sister HFS',
                                        'AB':'Adopted Brother AB',
                                        'AS':'Adopted Sister AS',
                                       }})[ii]



In [ ]:
ndarinventory=ndarinventory.drop(columns=['index'])
#p.shape
#c.shape
print(p.columns)
print(c.columns)

In [ ]:
#write out csv for validation
filePath=pathout+'/ndar_subject01.csv'

if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ndar_subject,1\n")
    ndarinventory.to_csv(f,index=False)



# Handedness

In [ ]:


#get handedness variables and ids from Redcap child
fieldlist=['hand1','hand2','hand3','hand4','hand5','hand6','hand7','hand8',
           'hand_total','iihandwr','iihandth','iihandsc','iihandto','iihandkn','iihandsp',
           'iihandbr','iihandma','iihandbo','iihandfk','iihandey','redcap_event_name','id','subject_id']
hcdchild=RedcapTable.get_table_by_name('hcpdchild').get_frame(fieldlist)
hcdchild.head()

In [ ]:
#hcdchild.value_counts('redcap_event_name')

childinventory3.loc[childinventory3["redcap_event_name"].isin(eventlist)].value_counts('redcap_event_name')

In [ ]:
#merge child handedness with inventory
hcdchildI=pd.merge(childinventory3.loc[childinventory3["redcap_event_name"].isin(eventlist)],hcdchild,left_on=['REDCap_id','redcap_event_name'],right_on= ['id','redcap_event_name'],how='left')
#drop teens
hcdchildI=hcdchildI.loc[~(hcdchildI.redcap_event_name=='visit_arm_1')]
hcdchildI.value_counts('redcap_event_name')
#hcdchildI.columns



In [ ]:
##hcd18 and older
list18=['iihandwr', 'iihandth', 'iihandsc', 'iihandto', 'iihandkn', 'iihandsp',
        'iihandbr', 'iihandma', 'iihandbo', 'iihandfk', 'iihandey','redcap_event_name','id']

hcpd18=RedcapTable.get_table_by_name('hcpd18').get_frame(list18)


In [ ]:
hcpd18I=pd.merge(childinventory3.loc[childinventory3["redcap_event_name"].isin(['visit_arm_1'])],hcpd18,left_on=['REDCap_id','redcap_event_name'],right_on= ['id','redcap_event_name'],how='left')
hcpd18I.value_counts('redcap_event_name')
#hcpd18I

In [ ]:
#concatenate hcdchild and hcd18...
hcpd18I.columns #18 and older dataframe
hcdchildI.columns #(child dataframe)
hcdtogether=pd.concat([hcpd18I,hcdchildI],axis=0,sort=True)
hcdtogether.to_csv('test.csv')
hcdtogether.columns

In [ ]:
#don't concat parents with children
#NO:concatenate parents with children
#parentinventory3['Source']='parent'
#hcdtogether['Source']='childteen'
#HCD=pd.concat([parentinventory3,hcdtogether],axis=0)
HCD=hcdtogether.copy()
HCD.columns
HCD.DB_Source.value_counts(dropna=False)


In [ ]:
hcdtogether2=HCD.copy()
hcdtogether2=hcdtogether2.rename(columns={"subject":"src_subject_id","M/F":"gender","nda_age":"interview_age",
                           "nda_interview_date":"interview_date","psuedo_guid":"subjectkey","redcap_event":"visit"})
hcdtogether2['DB_Source'].value_counts()

In [ ]:
flagsz=hcdtogether2.loc[hcdtogether2.race.isnull()==True]
#flags=flagsz[['subject_id','site_char','nda_interview_date']].copy()
#flags.loc[:,'reason']=pd.Series(("No race" for i in range(flags.shape[0])),index=flags.index)
flagsz
#hcdtogether2.loc[hcdtogether2.ethnic_group.isnull()==True]


In [ ]:
hcdtogether2.columns

In [ ]:
#hand
#last minute translation to valuess now requrested by NDA 4/8/2019
#spr;right;spl;left;both;np;ne; oh
#SPR=Strongly Prefer Right; SPL=Strongly Prefer Left; ne =not experienced; np=no preference; oh=sometimes uses other hand
#for ii in range(len(varlist)):
varlist=['hand1','hand2','hand3','hand4','hand5','hand6','hand7','hand8','iihandwr','iihandth','iihandsc','iihandto',
         'iihandkn','iihandsp','iihandbr','iihandma','iihandbo','iihandfk','iihandey']
varslim=['iihandwr','iihandth','iihandsc','iihandto',
         'iihandkn','iihandsp','iihandbr','iihandma','iihandbo','iihandfk','iihandey']
flagshand=pd.DataFrame(columns=['src_subject_id','site','reason'])
for ii in range(len(varslim)):
    #varhand=pd.DataFrame(hcdtogether2.loc[hcdtogether2[varlist[ii]]==''])
    varhand=pd.DataFrame((hcdtogether2.loc[(hcdtogether2[varslim[ii]].isnull()==True) & (hcdtogether2.hand_total.isnull()==True)]))
    varhand['reason']=varslim[ii]+' is missing'
    varhandslim=varhand[['src_subject_id','site','reason','interview_age']]
    flagshand=flagshand.append(varhandslim,sort=False)

flagshand#.reason.value_counts()

In [ ]:

varlistnoeye=['iihandwr','iihandth','iihandsc','iihandto','iihandkn','iihandsp','iihandbr','iihandma','iihandbo','iihandfk']
hcdtogetherolder = hcdtogether2.loc[(hcdtogether2.interview_age.astype(int) >= 132) & (hcdtogether2.src_subject_id != 'HCD0696265')].copy() #subject only came in at V1, so no need to extend logic in this case
#assign these older subjects missings to subject's median for score calculation, and then reset them to missing again later
#don't forget to reset
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD1807051'),'iihandsp']='4'
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD0036324'),'iihandfk']='5'
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD2415140'),'iihandfk']='4'
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD1502635'),'iihandto']='4'
hcdtogetherolder.loc[(hcdtogetherolder.interview_age==227) & (hcdtogetherolder.src_subject_id=='HCD2703751'),'iihandbo']='5'


#lightson 
hcdtogetherolder.to_csv('test.csv')

In [ ]:
test1=hcdtogetherolder.copy()
for var in varlistnoeye:
    test1[var]=test1.replace({var: {5:100, 4:50, 3:0, 2:-50, 1:-100}})[var]
    test1[var]=test1[var].astype(float)
    
test1['handvarsum']=test1[varlistnoeye].sum(axis=1)
test1['handvarsum']=test1['handvarsum']/10
test1=test1[varlistnoeye+['src_subject_id','interview_date','handvarsum']]
print(test1.columns)

test1.to_csv('test1.csv')
len(varlistnoeye)

test1[varlistnoeye].describe()
test1.head()

In [ ]:
test2=pd.merge(hcdtogetherolder,test1.drop(columns=varlistnoeye),on=['src_subject_id','interview_date'],how='left')
test2.handvarsum.describe()

In [ ]:
hcdtogetherolder=test2.copy()
hcdtogetherolder['HCP Handedness Score']=hcdtogetherolder.handvarsum
#set the score that are being calculated on missing values to zero
hcdtogetherolder.loc[(hcdtogetherolder['HCP Handedness Score'].round(0)==0) & (hcdtogetherolder.hand_total.isnull()==False)&(hcdtogetherolder.iihandwr.isnull()==True)][['hand_total','HCP Handedness Score','iihandwr']]
hcdtogetherolder.loc[(hcdtogetherolder['HCP Handedness Score'].round(0)==0)&(hcdtogetherolder.hand_total.isnull()==False)&(hcdtogetherolder.iihandwr.isnull()==True),'HCP Handedness Score']=''
hcdtogetherolder.columns

#now reset the hand set variables to missing
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD1807051'),'iihandsp']=''
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD0036324'),'iihandfk']=''
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD2415140'),'iihandfk']=''
hcdtogetherolder.loc[(hcdtogetherolder.visit=='V1') & (hcdtogetherolder.src_subject_id=='HCD1502635'),'iihandto']=''
hcdtogetherolder.loc[(hcdtogetherolder.interview_age==227) & (hcdtogetherolder.src_subject_id=='HCD2703751'),'iihandbo']=''


In [ ]:
#set the scores that are being calculated on missing values to zero
superlefts=['HCD2163644','HCD0382343']
#hcdtogetherolder.loc[~((hcdtogetherolder.hand_total>0) | ((hcdtogetherolder.src_subject_id.isin(superlefts)) & (hcdtogetherolder.hand6==0.0))),'hand_total']=''#.shape #interview_age.astype(int)>=132,'hand_total']=''
#hcdtogetherolder.loc[~((hcdtogetherolder.hand_total>0) | ((hcdtogetherolder.src_subject_id.isin(superlefts)) & (hcdtogetherolder.hand6==0.0)))][['hand_total','hand6','HCP Handedness Score']]
hcdtogetherolder.loc[(hcdtogetherolder.hand_total==0) & (~(hcdtogetherolder['HCP Handedness Score']=='') ),'hand_total']=''
hcdtogetherolder.loc[(hcdtogetherolder.hand_total==0) & (~(hcdtogetherolder['HCP Handedness Score']=='') ),'hand_total']=''
#also set the incorrect 0 score to missing. 
hcdtogetherolder.loc[(hcdtogetherolder.src_subject_id=='HCD1880162') & (hcdtogetherolder.interview_age==213),'hand_total']=''

In [ ]:
hand=hcdtogetherolder[['src_subject_id','gender', 'subjectkey', 'interview_date', 'interview_age','hand1','hand2','hand3','hand4','hand5','hand6','hand7','hand8','hand_total','iihandwr','iihandth','iihandsc',
            'iihandto', 'iihandkn','iihandsp', 'iihandbr', 'iihandma','iihandbo','iihandfk','iihandey','HCP Handedness Score']].copy()
iihandlist=['iihandwr','iihandth','iihandsc','iihandto', 'iihandkn','iihandsp', 'iihandbr', 'iihandma','iihandbo','iihandfk','iihandey']
newhandlist=['newiihandwr','newiihandth','newiihandsc','newiihandto', 'newiihandkn','newiihandsp', 'newiihandbr', 'newiihandma','newiihandbo','newiihandfk','newiihandey']
hand[iihandlist].head()

In [ ]:
for ii in range(len(iihandlist)):
    hand.loc[hand[iihandlist[ii]]=='',iihandlist[ii]]=-999
    #print(t.shape)            
    hand[iihandlist[ii]]=hand[iihandlist[ii]].fillna(-999).astype(int).round(0).astype(int).astype(str).replace('-999','')
    hand[newhandlist[ii]]=hand.replace({iihandlist[ii]:
                                       {'1':'left',
                                        '2':'spl',
                                        '3':'np',
                                        '4':'spr',
                                        '5':'right'}})[iihandlist[ii]]
    
handlist=['hand1','hand2','hand3','hand4','hand5','hand6','hand7','hand8']
newlist=['newhand1','newhand2','newhand3','newhand4','newhand5','newhand6','newhand7','newhand8']
for ii in range(len(handlist)):
    t=hand.loc[hand[handlist[ii]]=='']
    print(t.shape)            
    hand.loc[hand[handlist[ii]]=='',handlist[ii]]=-999
    hand[handlist[ii]]=hand[handlist[ii]].fillna(-999).round(0).astype(int).astype(str).replace('-999','')
    hand[newlist[ii]]=hand.replace({handlist[ii]:
                                       {'1':'right',
                                        '0':'left'}})[handlist[ii]]

hand.head()
hand.columns

In [ ]:



#lightson
hand.to_csv('test5.csv',index=False)
#print(hand.shape)
hand.head()
#hand.loc[hand['HCP Handedness Score']==0.0]

In [ ]:
print(hand.columns)

In [ ]:

hand.loc[(hand.newhand1.str.strip()=='') | (hand.newhand1.isnull()==True),'newhand1']=hand.newiihandwr
hand.loc[(hand.newhand3.str.strip()=='') | (hand.newhand3.isnull()==True),'newhand3']=hand.newiihandth
hand.loc[(hand.newhand7.str.strip()=='') | (hand.newhand7.isnull()==True),'newhand7']=hand.newiihandsc
hand.loc[(hand.newhand4.str.strip()=='') | (hand.newhand4.isnull()==True),'newhand4']=hand.newiihandto
hand.loc[(hand.newhand6.str.strip()=='') | (hand.newhand6.isnull()==True),'newhand6']=hand.newiihandsp


In [ ]:

#drop the hands (0s and 1s) and reame the newhands (lefts and rights) back to hands [long story]
#drop all the iihandlist variables (values 1::5)
#drop newiihandwr, newiihandth, newiihandsc, newiihandto, newiihandsp (values left,spl,np,spr,right) which are are now merged with hand1,hand3,hand7,hand4,hand6, respectively
#rename the remaining  newiihandkn, newiihandbr,newiihandma,newiihandbo,newiihandfk,newiihandey back to iihand counterparts

hand=hand.drop(columns=iihandlist+handlist+['newiihandwr', 'newiihandth', 'newiihandsc', 'newiihandto', 'newiihandsp'],axis=1)
dictnames=dict(zip(newlist,handlist))
hand=hand.rename(columns=dictnames)
hand=hand.rename(columns={'newiihandkn':'iihandkn','newiihandbr':'iihandbr','newiihandma':'iihandma','newiihandbo':'iihandbo','newiihandfk':'iihandfk','newiihandey':'iihandey'})

print(hand.shape)
hand.head()

In [ ]:
#4/19/2019 change again to element names to avoid confusion of aliases:
#rename the collection to original ndar data dictionary elements-note hand5 is a new element...not renamed
hand=hand.rename(columns={'hand1':'writing','hand2':'hammer','hand3':'throwing','hand4':'toothbrush','hand6':'spoon','hand7':'scissors','hand8':'hand_15_drink',
'iihandkn':'knife_no_fork','iihandbr':'broom','iihandma':'match','iihandbo':'box','iihandfk':'foot','iihandey':'eye'})
#hand['hcp_handedness_score']=hand['HCP Handedness Score']
hand=hand.rename(columns={'HCP Handedness Score':'hcp_handedness_score'})
hand.loc[hand.hcp_handedness_score.isnull()==True]
#hand.hcp_handedness_score=hand.hcp_handedness_score.fillna(-9999).astype(int).astype(str).str.replace('-9999','')
#print(hand.shape)


In [ ]:
hand2=hand.copy()

hand2.interview_age=hand.interview_age.astype(int)

hand2.loc[hand2.hcp_handedness_score=='','hcp_handedness_score']=-999#.fillna(-999).astype(int).astype(str).replace('-999','')
hand2.hcp_handedness_score=hand2.hcp_handedness_score.fillna(-999).astype(int).astype(str).replace('-999','')
hand2.interview_date=pd.to_datetime(hand2.interview_date).dt.strftime('%m/%d/%Y')
hand2.head()
#drop completely empty subject event
print(hand2.shape)
hand2=hand2.loc[~((hand2.src_subject_id=='HCD1880162') & (hand.interview_age==213))]
print(hand2.shape)


In [ ]:


filePath=pathout+'/edinburgh_hand01.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("edinburgh_hand,1\n")
    hand2.to_csv(f,index=False)
